# Using GPT for Product Recommendation Engines

Today we will be using SingleStore Kai™ for MongoDB, along with OpenAI, to put together a simple product recommendation engine in Python.

## API Keys

Before you get started, make sure you drop in your API key for OPENAI and your OPENAI ORG.

- You can find your `OPENAI_API_KEY` [here](https://platform.openai.com/account/api-keys), if you don't have one, generate a project specific one.
- You can find your `OPENAI_ORG` [here](https://platform.openai.com/account/org-settings) under `Organization ID`.

In [ ]:
OPENAI_API_KEY = ''
OPENAI_ORG = ''

## Import Dataset

First, we'll import a dataset into our database. Let's use [Open Library's Works](https://openlibrary.org/data/ol_dump_works_latest.txt.gz) dataset.

For the sake of brevity for the webinar, we've downloaded the extremely large dataset linked above and restricted it to just the Science Fiction novels using the following commands:

In [ ]:
wget https://openlibrary.org/data/ol_dump_works_latest.txt.gz
gunzip ol_dump_works_latest.txt
cat ol_dump_works_latest.txt | grep -i "science fiction" | grep -i "description" | cut -f5 > books_scifi.txt

#### Example Text Line

In [ ]:
{
    "created": {
        "type": "/type/datetime",
        "value": "2009-12-11T01:41:04.053897"
    },
    "latest_revision": 4,
    "description": "A science fiction suspense story. The setting starts on Earth, but then branches off to other realms with interesting\r\ncharacters.",
    "key": "/works/OL9889262W",
    "title": "Dark Paladin",
    "authors": [
        {
            "type": {
                "key": "/type/author_role"
            },
            "author": {
                "key": "/authors/OL3871508A"
            }
        }
    ],
    "type": {
        "key": "/type/work"
    },
    "last_modified": {
        "type": "/type/datetime",
        "value": "2012-07-11T20:19:14.469842"
    },
    "covers": [
        2940133
    ],
    "revision": 4
}

### Download Dataset

I have hosted the slimmed down dataset on the [GitHub Repo](https://github.com/singlestore-labs/webinar-code-examples/tree/main/kai-product-rec) for this webinar. We'll download it from there.

Using the `requests` library, we will download the txt file, iterate through it in chunks, writing those chunks to a local file.

Below, you'll see us setting the output of the `download_file()` function to the variable `local_dataset`.

In [ ]:
import requests

dataset_url = 'https://raw.githubusercontent.com/singlestore-labs/webinar-code-examples/main/kai-product-rec/books_scifi.txt'

def download_file(dataset_url):
    local_filename = dataset_url.split('/')[-1]
    with requests.get(dataset_url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk: 
                f.write(chunk)
    return local_filename

local_dataset = download_file(dataset_url)

### Read text file into variable

The file created in the previous step will now be opened and read line-by-line into the variable `data`.

In [ ]:
file = open(local_dataset)
data = file.readlines()
file.close()

### Track Token Usage

Querying OpenAI a bunch of times can get expensive, so you'll want to keep track of your token usage. In our case, I looked at the [Pricing Page](https://openai.com/pricing). specifically for the model that we will be using to create our vectors (`text-embedding-ada-002`).

The function below will take the dollar limit you set for the project and do the math to determine the max number of tokens you can expend, while the function (`budget_status()` will let us know throughout our project how we're doing on token usage.

In [ ]:
cost_per_1k = 0.0001
dollar_limit = 20.00
budget_tokens = (dollar_limit / cost_per_1k) * 1000
token_usage = 0

def budget_status(token_usage):
    if budget_tokens > token_usage:
        return 'ok'
    else:
        return 'spent'

### Create SQL Table

Here we are creating the table to store our books in. We will have an auto incrementing `_id` as the primary key, while storing the book title and the embedding generated from OpenAI.

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS products (
  _id INT AUTO_INCREMENT PRIMARY KEY,
  title VARCHAR(255) NOT NULL,
  embedding BLOB NOT NULL
);

### Create Embeddings and Load into S2

This is a fairly complex process, so we'll break everything out as best as we can here. Here are the high-level steps we need to cover:

1. Install `openai` and import required libraries. Additionally set some default configuration settings and create the database connection.

> Note: `connection_url` is unique to SingleStore Notebooks, as it's a variable that contains the connection string for your databse.

2. Create the functions needed to query OpenAI for the Embeddings and to Write to the Database.
3. Loop through the books dataset (which is stored in `data`), calling for the creation of embedding, then store it to the database.

#### Step 1: Install and import libraries, adjust configuration

In [ ]:
!pip install openai

import openai
import ast
from sqlalchemy import *

openai.organization = OPENAI_ORG
openai.api_key = OPENAI_API_KEY

conn = create_engine(connection_url)
update_interval = 500 # How often to update you in the terminal of status

model_id = 'text-embedding-ada-002'

#### Step 2: Create the functions to query OpenAI and Write to Database

In [ ]:
ds_with_embeddings = []
total_items = len(data)

def request_embedding(text, token_usage):
    
    budget = budget_status(token_usage)
    
    if budget == 'ok':
        #print('Budget status: OK\nTokens: {}/{}'.format(token_usage,budget_tokens))
        try:
            if OPENAI_API_KEY:
                response = openai.Embedding.create(input=text,model=model_id)
                embedding = response['data'][0]['embedding']
                tokens = response['usage']['total_tokens']
                status = 'success'
                #print(embedding)
                return embedding,tokens,status
            else:
                print('You need to set your OpenAI API Key to the variable OPENAI_API_KEY')
        except Exception as e:
            print(e)
            embedding = ''
            tokens = 0
            status = 'failed'
            return embedding,tokens,status
    else:
        print('Budget Spent: {}/{}'.format(token_usage,budget_tokens))
        embedding = ''
        tokens = 0
        status = 'budget_spent'
        return embedding,tokens,status

def write_to_db(data):
    keys = ["title", "embedding" ];
    query = "INSERT INTO products (title, embedding) VALUES (%s, JSON_ARRAY_PACK_F32(%s))"
    
    try:
        with conn:
            conn.execute(query, (data[keys[0]].replace("'",""), str(data[keys[1]])))
            print("Wrote item")
    except Exception as e:
        print(e)

#### Step 4: Loop through dataset requesting embedding, append embedding to dataset, write dataset to database

In [ ]:
loop_counter = 0
print('Requesting embeddings. I will update you every {} embeddings.'.format(str(update_interval))
for b in data:
    try:
        embedding,tokens,status = request_embedding(b, token_usage)
        if status != 'failed' and status != 'budget_spent':
            book = ast.literal_eval(b)
            book['embedding'] = embedding
            write_to_db(book)
            token_usage += tokens
            #print('Completed {}/{}'.format(len(ds_with_embeddings),total_items))
            loop_counter += 1
            if loop_counter == update_interval:
                print('Completed {}/{}'.format(len(ds_with_embeddings),total_items))
                print('Token usage: {}/{}'.format(token_usage,budget_tokens))
                loop_counter = 0
        elif status == 'budget_spent':
            print('Getting embedding failed because the budget is spent.')
        else:
            print('Getting embedding for this book failed:\n{}'.format(b))
    except Exception as e:
        print(e)
        
conn.close()

In [ ]:
query = 'The Martian'



sql_query = 'SELECT title FROM products WHERE EUCLIDEAN_DISTANCE(vector, JSON_ARRAY_PACK('query')) ;

SELECT EUCLIDEAN_DISTANCE(vector, JSON_ARRAY_PACK('[5.9,3,5.1,1.8]')) AS euclidean_distance, title
FROM products
ORDER BY euclidean_distance
LIMIT 5;

